# Transition-based arc-eager unlabeled dependency parser for Estonian

## Read the data

To prepare the training data for the parser, we will be using:
* [UD corpus for Estonian](https://github.com/UniversalDependencies/UD_Estonian-EDT)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

PATH = "UD_Estonian-EDT"

with open(PATH + "/et_edt-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [2]:
tree = trees[0]
print(tree[0])

OrderedDict([('id', 1), ('form', 'Iga'), ('lemma', 'iga'), ('upostag', 'DET'), ('xpostag', 'P'), ('feats', OrderedDict([('Case', 'Nom'), ('Number', 'Sing'), ('PronType', 'Tot')])), ('head', 3), ('deprel', 'det'), ('deps', None), ('misc', None)])


In [3]:
# Every ninth crown came from mysterious people.
" ".join([node["form"] for node in tree])

'Iga üheksas kroon tuli salapärastelt isikutelt .'

In [4]:
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

Iga <-- kroon
üheksas <-- kroon
kroon <-- tuli
tuli <-- root
salapärastelt <-- isikutelt
isikutelt <-- tuli
. <-- tuli


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [5]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [6]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [7]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'),
                    ('upostag', 'ROOT'), ('xpostag', None),
                    ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['Iga_1', 'üheksas_2', 'kroon_3', 'tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'Iga_1']
Queue: ['üheksas_2', 'kroon_3', 'tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'Iga_1', 'üheksas_2']
Queue: ['kroon_3', 'tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: []
Actions.LEFT
Stack: ['ROOT_0', 'Iga_1']
Queue: ['kroon_3', 'tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: [(2, 3)]
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['kroon_3', 'tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: [(2, 3), (1, 3)]
Actions.SHIFT
Stack: ['ROOT_0', 'kroon_3']
Queue: ['tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: [(2, 3), (1, 3)]
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['tuli_4', 'salapärastelt_5', 'isikutelt_6', '._7']
Relations: [(2, 3), (1, 3), (3, 4)]
Actions.RIGHT
Stack: ['ROOT_0', 'tuli_4']
Queue: ['salapärastelt_5', 'isikutelt_6', '

In [8]:
trace_actions(trees[1])

Stack: ['ROOT_0']
Queue: ['Eesti_1', 'Ekspressi_2', 'teada_3', 'on_4', 'Eesti_5', 'Pank_6', 'uurinud_7', 'Hansapanga_8', 'tehinguid_9', ',_10', 'mis_11', 'toimusid_12', 'kaks_13', 'aastat_14', 'tagasi_15', 'suvel_16', 'ja_17', 'mille_18', 'käigus_19', 'voolas_20', 'panka_21', 'ligi_22', 'miljardi_23', 'krooni_24', 'ulatuses_25', 'kahtlast_26', 'raha_27', '._28']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'Eesti_1']
Queue: ['Ekspressi_2', 'teada_3', 'on_4', 'Eesti_5', 'Pank_6', 'uurinud_7', 'Hansapanga_8', 'tehinguid_9', ',_10', 'mis_11', 'toimusid_12', 'kaks_13', 'aastat_14', 'tagasi_15', 'suvel_16', 'ja_17', 'mille_18', 'käigus_19', 'voolas_20', 'panka_21', 'ligi_22', 'miljardi_23', 'krooni_24', 'ulatuses_25', 'kahtlast_26', 'raha_27', '._28']
Relations: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['Ekspressi_2', 'teada_3', 'on_4', 'Eesti_5', 'Pank_6', 'uurinud_7', 'Hansapanga_8', 'tehinguid_9', ',_10', 'mis_11', 'toimusid_12', 'kaks_13', 'aastat_14', 'tagasi_15', 'suvel_16', 'ja_17',

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [9]:
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
    if len(stack) > 1:
        features["s1-tag"] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    return features

## Prepare train and test data

In [10]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [11]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)

Number of words: 7
Number of actions: 15
List of actions taken: ['shift', 'shift', 'left', 'left', 'shift', 'left', 'right', 'shift', 'left', 'right', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'Iga', 'q0-lemma': 'iga', 'q0-tag': 'DET', 'q1-word': 'üheksas', 'q1-tag': 'ADJ', 'q2-tag': 'NOUN', 'q3-tag': 'VERB'}
{'s0-word': 'Iga', 's0-lemma': 'iga', 's0-tag': 'DET', 's1-tag': 'ROOT', 'q0-word': 'üheksas', 'q0-lemma': 'üheksas', 'q0-tag': 'ADJ', 'q1-word': 'kroon', 'q1-tag': 'NOUN', 'q2-tag': 'VERB', 'q3-tag': 'ADJ'}
{'s0-word': 'üheksas', 's0-lemma': 'üheksas', 's0-tag': 'ADJ', 's1-tag': 'DET', 'q0-word': 'kroon', 'q0-lemma': 'kroon', 'q0-tag': 'NOUN', 'q1-word': 'tuli', 'q1-tag': 'VERB', 'q2-tag': 'ADJ', 'q3-tag': 'NOUN'}
{'s0-word': 'Iga', 's0-lemma': 'iga', 's0-tag': 'DET', 's1-tag': 'ROOT', 'q0-word': 'kroon', 'q0-lemma': 'kroon', 'q0-tag': 'NOUN', 'q1-word': 'tuli', 'q1-tag': 'VERB', 'q2-tag': 'ADJ'

In [12]:
train_features, train_labels = [], []
for tree in trees:
    tree_features, tree_labels = get_data(tree)
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

706628 706628


In [13]:
# Test data

with open(PATH + "/et_edt-ud-dev.conllu", "r") as f:
    data = f.read()
test_trees = parse(data)

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data(tree)
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

92389 92389


## Train a classifier

In [14]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [15]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  284211


In [16]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

print(len(train_features_vectorized.toarray()),
      len(test_features_vectorized.toarray()))

706628 92389


In [17]:
lrc = LogisticRegression(random_state=42, solver="saga",
                         multi_class="multinomial", max_iter=1000,
                         verbose=1)
lrc.fit(train_features_vectorized, train_labels)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 507 epochs took 600 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.0min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='saga',
          tol=0.0001, verbose=1, warm_start=False)

In [18]:
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

              precision    recall  f1-score   support

        left       0.81      0.84      0.83     24155
      reduce       0.86      0.84      0.85     23602
       right       0.82      0.78      0.80     20095
       shift       0.82      0.84      0.83     24537

   micro avg       0.83      0.83      0.83     92389
   macro avg       0.83      0.83      0.83     92389
weighted avg       0.83      0.83      0.83     92389



## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [19]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if log:
                print("Stack:", [i["form"]+"_"+str(i["id"])
                                 for i in stack])
                print("Queue:", [i["form"]+"_"+str(i["id"])
                                 for i in queue])
                print("Relations:", relations)
                print(action)
                print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [20]:
print([node["form"] for node in test_trees[0]])
print(dep_parse(test_trees[0], lrc, vec))
print([(node["id"], node["head"]) for node in test_trees[0]])

['Aga', 'mulle', 'tundub', ',', 'et', 'kogu', 'maailm', 'ootab', 'muusikamaailmalt', 'midagi', 'erutavalt', 'uut', 'minimalismi', 'kõrvale', '.']
Stack: ['ROOT_0']
Queue: ['Aga_1', 'mulle_2', 'tundub_3', ',_4', 'et_5', 'kogu_6', 'maailm_7', 'ootab_8', 'muusikamaailmalt_9', 'midagi_10', 'erutavalt_11', 'uut_12', 'minimalismi_13', 'kõrvale_14', '._15']
Relations: []
shift
Stack: ['ROOT_0', 'Aga_1']
Queue: ['mulle_2', 'tundub_3', ',_4', 'et_5', 'kogu_6', 'maailm_7', 'ootab_8', 'muusikamaailmalt_9', 'midagi_10', 'erutavalt_11', 'uut_12', 'minimalismi_13', 'kõrvale_14', '._15']
Relations: []
shift
Stack: ['ROOT_0', 'Aga_1', 'mulle_2']
Queue: ['tundub_3', ',_4', 'et_5', 'kogu_6', 'maailm_7', 'ootab_8', 'muusikamaailmalt_9', 'midagi_10', 'erutavalt_11', 'uut_12', 'minimalismi_13', 'kõrvale_14', '._15']
Relations: []
left
Stack: ['ROOT_0', 'Aga_1']
Queue: ['tundub_3', ',_4', 'et_5', 'kogu_6', 'maailm_7', 'ootab_8', 'muusikamaailmalt_9', 'midagi_10', 'erutavalt_11', 'uut_12', 'minimalismi_13', 

In [21]:
print([node["form"] for node in test_trees[3]])
print(dep_parse(test_trees[3], lrc, vec))
print([(node["id"], node["head"]) for node in test_trees[3]])

['Godard', 'on', 'märkinud', ',', 'et', 'Vermeer', 'on', 'parimaid', 'fotograafe', 'kakssada', 'aastat', 'enne', 'fotokunsti', ',', 'sest', 'Vermeer', 'huvitus', 'tohutult', 'oskusest', 'vaadata', ',', 'ajahetke', 'jäädvustamisest', '.']
Stack: ['ROOT_0']
Queue: ['Godard_1', 'on_2', 'märkinud_3', ',_4', 'et_5', 'Vermeer_6', 'on_7', 'parimaid_8', 'fotograafe_9', 'kakssada_10', 'aastat_11', 'enne_12', 'fotokunsti_13', ',_14', 'sest_15', 'Vermeer_16', 'huvitus_17', 'tohutult_18', 'oskusest_19', 'vaadata_20', ',_21', 'ajahetke_22', 'jäädvustamisest_23', '._24']
Relations: []
shift
Stack: ['ROOT_0', 'Godard_1']
Queue: ['on_2', 'märkinud_3', ',_4', 'et_5', 'Vermeer_6', 'on_7', 'parimaid_8', 'fotograafe_9', 'kakssada_10', 'aastat_11', 'enne_12', 'fotokunsti_13', ',_14', 'sest_15', 'Vermeer_16', 'huvitus_17', 'tohutult_18', 'oskusest_19', 'vaadata_20', ',_21', 'ajahetke_22', 'jäädvustamisest_23', '._24']
Relations: []
shift
Stack: ['ROOT_0', 'Godard_1', 'on_2']
Queue: ['märkinud_3', ',_4', 'et

Stack: ['ROOT_0', 'märkinud_3']
Queue: ['huvitus_17', 'tohutult_18', 'oskusest_19', 'vaadata_20', ',_21', 'ajahetke_22', 'jäädvustamisest_23', '._24']
Relations: [(2, 3), (1, 3), (3, 0), (8, 9), (7, 9), (6, 9), (5, 9), (4, 9), (9, 3), (10, 11), (11, 3), (12, 13), (13, 3), (16, 17), (15, 17), (14, 17)]
right
Stack: ['ROOT_0', 'märkinud_3', 'huvitus_17']
Queue: ['tohutult_18', 'oskusest_19', 'vaadata_20', ',_21', 'ajahetke_22', 'jäädvustamisest_23', '._24']
Relations: [(2, 3), (1, 3), (3, 0), (8, 9), (7, 9), (6, 9), (5, 9), (4, 9), (9, 3), (10, 11), (11, 3), (12, 13), (13, 3), (16, 17), (15, 17), (14, 17), (17, 3)]
shift
Stack: ['ROOT_0', 'märkinud_3', 'huvitus_17', 'tohutult_18']
Queue: ['oskusest_19', 'vaadata_20', ',_21', 'ajahetke_22', 'jäädvustamisest_23', '._24']
Relations: [(2, 3), (1, 3), (3, 0), (8, 9), (7, 9), (6, 9), (5, 9), (4, 9), (9, 3), (10, 11), (11, 3), (12, 13), (13, 3), (16, 17), (15, 17), (14, 17), (17, 3)]
left
Stack: ['ROOT_0', 'märkinud_3', 'huvitus_17']
Queue: ['o

In [22]:
total, tp = 0, 0
for tree in test_trees:
    golden = [(node["id"], node["head"]) for node in tree]
    predicted = dep_parse(tree, lrc, vec, log=False)
    total += len(tree)
    tp += len(set(golden).intersection(set(predicted)))

print("Total:", total)
print("Correctly defined:", tp)
print("UAS:", round(tp/total, 2))

Total: 44632
Correctly defined: 30272
UAS: 0.68


## Try on a new sentence

In [26]:
# I like turtles.
new_sentence = "Mulle meeldivad kilpkonnad .".split()
new_lemmas = "mina meeldima kilpkonn .".split()
new_pos = "PRON VERB NOUN PUNCT".split()

In [33]:
new_conll = []
for i, j in enumerate(zip(new_sentence, new_lemmas, new_pos)):
    word, lemma, tag = j
    new_conll.append(OrderedDict([('id', i+1), ('form', word),
                                  ('lemma', lemma), ('upostag', tag),
                                  ('xpostag', None), ('feats', None),
                                  ('head', None), ('deprel', None),
                                  ('deps', None), ('misc', None)]))

In [34]:
new_conll

[OrderedDict([('id', 1),
              ('form', 'Mulle'),
              ('lemma', 'mina'),
              ('upostag', 'PRON'),
              ('xpostag', None),
              ('feats', None),
              ('head', None),
              ('deprel', None),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 2),
              ('form', 'meeldivad'),
              ('lemma', 'meeldima'),
              ('upostag', 'VERB'),
              ('xpostag', None),
              ('feats', None),
              ('head', None),
              ('deprel', None),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 3),
              ('form', 'kilpkonnad'),
              ('lemma', 'kilpkonn'),
              ('upostag', 'NOUN'),
              ('xpostag', None),
              ('feats', None),
              ('head', None),
              ('deprel', None),
              ('deps', None),
              ('misc', None)]),
 OrderedDict([('id', 4),
              ('

In [35]:
dep_parse(new_conll, lrc, vec)

Stack: ['ROOT_0']
Queue: ['Mulle_1', 'meeldivad_2', 'kilpkonnad_3', '._4']
Relations: []
shift
Stack: ['ROOT_0', 'Mulle_1']
Queue: ['meeldivad_2', 'kilpkonnad_3', '._4']
Relations: []
left
Stack: ['ROOT_0']
Queue: ['meeldivad_2', 'kilpkonnad_3', '._4']
Relations: [(1, 2)]
right
Stack: ['ROOT_0', 'meeldivad_2']
Queue: ['kilpkonnad_3', '._4']
Relations: [(1, 2), (2, 0)]
right
Stack: ['ROOT_0', 'meeldivad_2', 'kilpkonnad_3']
Queue: ['._4']
Relations: [(1, 2), (2, 0), (3, 2)]
reduce
Stack: ['ROOT_0', 'meeldivad_2']
Queue: ['._4']
Relations: [(1, 2), (2, 0), (3, 2)]
right


[(1, 2), (2, 0), (3, 2), (4, 2)]

### Your turn
* Add a dependency label classifier to finalize the parser. Measure the LAS (labelled attachment score).
* Add more features to improve the quality of the parser. Explore top features to understand if your classifiers are learning properly.
* Add beam search or a non-deterministic oracle.
* Compare your results with the parser from [estnltk](https://github.com/estnltk/estnltk).
* Use [estnltk](https://github.com/estnltk/estnltk) for part-of-speech tagging and lemmatization of new sentences. Use your parser on new sentences.